# Anime Recommendation Algorithm
This model uses a content-based algoritm to recommend animes based on genre and user ratings.

## Imports & Preprocessing

Before getting started with data preperation, we must import the datasets and tools necessary for this lab.

In [108]:
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

In [109]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_24617fd1522c4b708f743238fde6b4b4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vptwfiLBRO480cz0C3Iup75vm2bDMVyDKrs6iLrON-tT',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_24617fd1522c4b708f743238fde6b4b4.get_object(Bucket='animerecommendationalgorithm-donotdelete-pr-xqralsrto7kg1c',Key='anime.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [110]:
body = client_24617fd1522c4b708f743238fde6b4b4.get_object(Bucket='animerecommendationalgorithm-donotdelete-pr-xqralsrto7kg1c',Key='rating.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


## Data Cleaning

In this stage, we will create the anime dataframe as a copy from the original df but with only necessary columns.

In [111]:
anime_df = df_data_1[['anime_id','name','genre']]
anime_df.head(5)

,anime_id,name,genre
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural"
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S..."


Split the values in 'genre' to a list of genres to simplify future use.

In [112]:
anime_df['genre'] = anime_df.genre.str.split(',')
anime_df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,anime_id,name,genre
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]"
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,..."
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur..."
3,9253,Steins;Gate,"[Sci-Fi, Thriller]"
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur..."


We will use the one-hot encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. <br><br>
In other words, turning categorical data into numerical.

In [113]:
# Copying the anime dataframe into a new one since we won't need to use the genre information in our first case.
animeWithGenre_df = anime_df.copy()

# For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in anime_df.iterrows():
    for genre in row['genre']:
        animeWithGenre_df.at[index, genre] = 1

TypeError: 'float' object is not iterable

In [114]:
# Filling in the NaN values with 0 to show that an anime doesn't have that column's genre
animeWithGenre_df = animeWithGenre_df.fillna(0)

In [115]:
animeWithGenre_df.head()

,anime_id,name,genre,Drama,Romance,School,Supernatural,Action,Adventure,Drama,...,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
animeWithGenre_df.dtypes

anime_id            int64
name               object
genre              object
Drama             float64
 Romance          float64
 School           float64
 Supernatural     float64
Action            float64
 Adventure        float64
 Drama            float64
 Fantasy          float64
 Magic            float64
 Military         float64
 Shounen          float64
 Comedy           float64
 Historical       float64
 Parody           float64
 Samurai          float64
 Sci-Fi           float64
Sci-Fi            float64
 Thriller         float64
Comedy            float64
 Sports           float64
 Super Power      float64
 Space            float64
 Slice of Life    float64
 Mecha            float64
Adventure         float64
 Music            float64
 Mystery          float64
                   ...   
 Josei            float64
Josei             float64
Military          float64
Romance           float64
 Shounen Ai       float64
 Game             float64
Demons            float64
 Ecchi      

In [117]:
ratings_df = df_data_2
ratings_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [118]:
ratings_df.shape

(7813737, 3)

In [119]:
anime_df.shape

(12294, 3)

## Content-Based Recommendation System

Add user input (inital anime preferences)

In [136]:
userInput = [
            {'name':'Lupin III', 'rating':9.55},
            {'name':'Detective Conan', 'rating':8.83},
            {'name':'Code Geass: Hangyaku no Lelouch', 'rating':8.23},
            {'name':'GintamaÂ°', 'rating':9.77},
            {'name':'Medaka Box Abnormal', 'rating':8.09}
            ]
inputAnime = pd.DataFrame(userInput)
inputAnime

,name,rating
0,Lupin III,9.55
1,Detective Conan,8.83
2,Code Geass: Hangyaku no Lelouch,8.23
3,GintamaÂ°,9.77
4,Medaka Box Abnormal,8.09


Now, we will add anime_id to the input user dataframe.
<br><br>
With the input complete, let's extract then input anime's ID's from the anime dataframe and add them to it.
<br><br>
We can acieve this by first filtering out the rows that contain the input movie's title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save on memory space.

In [137]:
# Filtering out the movies by title
inputID = anime_df[anime_df['name'].isin(inputAnime['name'].tolist())]

# Then merging it so we can get the animeID. It's implicitly merging it by title.
inputAnime = pd.merge(inputID, inputAnime)

# Dropping genre column to save on memory
inputAnime = inputAnime.drop('genre', 1)

#Final input dataframe
inputAnime

,anime_id,name,rating
0,1575,Code Geass: Hangyaku no Lelouch,8.23
1,235,Detective Conan,8.83
2,1412,Lupin III,9.55
3,14527,Medaka Box Abnormal,8.09


Learn the input's preferences by getting the subset of movies that the input has watched from the dataframe containing genres defined with binary values.

In [138]:
# Filter out the anime from the input
userAnime = animeWithGenre_df[animeWithGenre_df['anime_id'].isin(inputAnime['anime_id'].tolist())]
userAnime

,anime_id,name,genre,Drama,Romance,School,Supernatural,Action,Adventure,Drama,...,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen
19,1575,Code Geass: Hangyaku no Lelouch,"[Action, Mecha, Military, School, Sci-Fi, ...",0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,235,Detective Conan,"[Adventure, Comedy, Mystery, Police, Shounen]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
936,1412,Lupin III,"[Action, Adventure, Comedy, Shounen]",0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1209,14527,Medaka Box Abnormal,"[Action, Comedy, Ecchi, Martial Arts, Scho...",0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Reset index and drop anime_id, name and genre.

In [139]:
# Reset index
userAnime = userAnime.reset_index(drop=True)
# Drop anime_id, name and genre
userGenreTable = userAnime.drop('anime_id', 1).drop('name', 1).drop('genre', 1)
userGenreTable

,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,...,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Learn input preferences...

Use the input's review and multiply thme into the input genre table and then sum up the resulting table by column.

In [140]:
inputAnime['rating']

0    8.23
1    8.83
2    9.55
3    8.09
Name: rating, dtype: float64

In [141]:
# Dot product to get weights
userProfile = userGenreTable.transpose().dot(inputAnime['rating'])
# Display user profile
userProfile

Drama              0.00
 Romance           0.00
 School           16.32
 Supernatural      0.00
Action            25.87
 Adventure         9.55
 Drama             0.00
 Fantasy           0.00
 Magic             0.00
 Military          8.23
 Shounen          26.47
 Comedy           26.47
 Historical        0.00
 Parody            0.00
 Samurai           0.00
 Sci-Fi            8.23
Sci-Fi             0.00
 Thriller          0.00
Comedy             0.00
 Sports            0.00
 Super Power      16.32
 Space             0.00
 Slice of Life     0.00
 Mecha             8.23
Adventure          8.83
 Music             0.00
 Mystery           8.83
 Seinen            0.00
Fantasy            0.00
 Martial Arts      8.09
                  ...  
 Josei             0.00
Josei              0.00
Military           0.00
Romance            0.00
 Shounen Ai        0.00
 Game              0.00
Demons             0.00
 Ecchi             8.09
Dementia           0.00
 Harem             0.00
Music           

Extract the genre table from the original dataframe.

In [142]:
# Get the genres of every movie in our original dataframe
genreTable = animeWithGenre_df.set_index(animeWithGenre_df['anime_id'])

# Drop unnecessary information
genreTable = genreTable.drop('anime_id', 1).drop('name', 1).drop('genre', 1)
genreTable.head()

,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,...,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen
anime_id,,,,,,,,,,,,,,,,,,,,,
32281,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
genreTable.shape

(12294, 69)

Take the weighted average of every anime based on the input profile and recommend the top twenty movies that most satisfy it.

In [144]:
# Multiple genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

anime_id
32281    0.086643
5114     0.372266
28977    0.462094
9253     0.000000
9969     0.462094
dtype: float64

In [145]:
# Sort recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

anime_id
11761    0.677585
14527    0.677585
12231    0.642387
1397     0.600181
573      0.599437
dtype: float64

In [148]:
# Final recommendation table
anime_df.loc[anime_df['anime_id'].isin(recommendationTable_df.head(20).keys())]

,anime_id,name,genre
178,31964,Boku no Hero Academia,"[Action, Comedy, School, Shounen, Super Po..."
206,813,Dragon Ball Z,"[Action, Adventure, Comedy, Fantasy, Marti..."
352,8740,One Piece Film: Strong World Episode 0,"[Action, Adventure, Comedy, Fantasy, Shoun..."
515,22777,Dragon Ball Kai (2014),"[Action, Adventure, Comedy, Fantasy, Marti..."
588,6033,Dragon Ball Kai,"[Action, Adventure, Comedy, Fantasy, Marti..."
1209,14527,Medaka Box Abnormal,"[Action, Comedy, Ecchi, Martial Arts, Scho..."
1409,25389,Dragon Ball Z Movie 15: Fukkatsu no F,"[Action, Adventure, Comedy, Fantasy, Marti..."
1432,25,Sunabouzu,"[Action, Adventure, Comedy, Ecchi, Sci-Fi,..."
1762,573,Saber Marionette J,"[Action, Adventure, Comedy, Drama, Harem, ..."
1795,1094,One Piece: Umi no Heso no Daibouken-hen,"[Action, Adventure, Comedy, Fantasy, Shoun..."
